In [2]:
from dotenv import load_dotenv

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

from typing import Annotated, TypedDict

#### .env 로드
load_dotenv()

True

In [3]:
#### GPT-4o-mini 설정
gpt4o_mini = ChatOpenAI(
    model_name="gpt-4o-mini",  # GPT-4o-mini에 해당하는 모델명
    temperature=0.7,
    max_tokens=150,
)

#### GPT-4o 설정
gpt4o = ChatOpenAI(
    model_name="gpt-4o",  # GPT-4o에 해당하는 모델명
    temperature=0.7,
    max_tokens=300,
)

C:\Users\maeng\AppData\Local\Temp\ipykernel_18120\3298326656.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  gpt4o_mini = ChatOpenAI(


In [4]:
#### GPT-4o-mini 사용 예시
response_mini = gpt4o_mini.invoke([HumanMessage(content="안녕 넌 어때?")])
print(response_mini.content, '\n\n')

안녕하세요! 저는 잘 지내고 있어요. 당신은 어떠신가요? 도움이 필요하시면 언제든지 말씀해 주세요! 




### 사용자 질의의 문법
1. 정형화된 문구로 변환


### 사용자 이전 질의와의 관련성 확인 필요 유무
1. 이전 질의와의 관련성 확인
    - 몇 개까지 관련성을 확인 할 것인가 --> 메모리 노드 필요


### 사용자 질의의 context
1. 특정 context에 관련된 문제를 찾는가
    - 언어 (SQL, python, ...)
    - 난이도
    
2. 특정 사이트에 있는 문제를 찾는가
    - 코드프로그래머스
    - 백준


### 코드 평가 결과 명령
1. 평가 방법을 찾지 못했는가
2. 평가 결과가 정확한가
3. DB, web에서 데이터를 찾아야하는가


### 사용자의 다음 질의 준비
1. 다음 질의 예측을 몇개나 할것인가
2. 다음 질의 예측을 위한 이전 질의를 몇개까지 사용할 것인가


### 메모리 노드
추후에

3. 관련성
        - 이전에 사용자가 질문했던 내용들을 고려해서 답변해줘
        
4. 사용자의 질문 예측
    - 사용자가 요청한 문제들을 토대로 다음에 요청할 예상 질문을 예측하고 답변을 미리 준비 해줘

In [ ]:
######## 그래프 상태 값
class MyState(TypedDict): # 그래프의 상태를 정의하는 클래스
    relevance_score : float
    previous_bool : bool # 하나의 값으로?
    accuracy_bool : bool # 클로이드?
    
    user_prompt : str
    generated_problem : str
    user_answer : str
    
    # messages: Annotated[list, add_messages] # 이벤트 내역 출력 용

graph_builder = StateGraph(MyState) # StateGraph 인스턴스 생성



######## 노드
def test_node_01(state):
    user_prompt = state["user_prompt"]
    # system_prompt = f'사용자의 질의 :{user_prompt}'
    system_prompt = f'''
    
    너는 사용자에게 코딩 문제를 제공하는 모델이야. 사용자의 요구에 따라 적절한 문제를 사용자에게 내줘.
    또한 밑에 있는 사항을 고려해줘
        
    - 사용자가 컴퓨터 언어를 특정한 경우 해당 언어에 대한 문제를 생성해줘
    
    - 사용자가 원하는 문제의 난이도가 있을 경우, 적절한 난이도를 고려해서 문제를 생성해줘
    
    - 테이블과 같은 텍스트로만 보기 어려운 내용은 md 형식으로 사용자가 보기 편하게 출력해줘
    
    - 사용자의 문제 풀이를 평가할 수 있도록 정확한 답이 있는 문제를 내줘
    
    
    - 예시 1 :
        사용자 질문 : python 기초 문제를 내줘
        문제 생성 : 정수 3과 7을 변수에 할당하고 두 수의 합을 구하는 코드를 작성하세요.
        
    - 예시 2 :
        사용자 질문 : python 쉬운 문제를 내줘
        문제 생성 : 정수 8과 12을 변수에 할당하고 두 수의 곱을 구하는 코드를 작성하세요.
        
    - 예시 3 :
    사용자 질문 : sql 기초 문제 내줘
    문제 생성 : 
        | employee_id | gender | age |
        |-----|---|----|
        | 001 | M | 20 |
        | 002 | M | 33 |
        | 003 | W | 52 |
        | 004 | W | 26 |
        | 002 | M | 33 |
        
        다음 테이블에서 남자인 사람들을 조회하세요.
    
    
    
    사용자의 요청 :{user_prompt}
    
    '''
    
    return {"generated_problem": gpt4o_mini.invoke([system_prompt])} # convert user_question


def test_node_02(state):
    user_answer = input()
    return {"user_answer": user_answer} # convert user_question


#### 노드 선언
graph_builder.add_node("test_node_01", test_node_01)
graph_builder.add_node("test_node_02", test_node_02)



######## 엣지
#### 엣지 선언
graph_builder.add_edge(START, "test_node_01")
graph_builder.add_edge("test_node_01", "test_node_02")
graph_builder.add_edge("test_node_02", END)



######## 그래프
#### 그래프 컴파일
graph = graph_builder.compile()

#### 그래프 실행
relevance_score = 0.0
previous_bool = False
accuracy_bool = False

user_prompt = 'sql 심화 문제를 내줘'
generated_problem = ''
user_answer = ''



relevance = 0.0
previous_bool = False

accuracy_bool = False

result = graph.invoke({"relevance_score": relevance_score,
                    # "messages": user_prompt,
                    "previous_bool": previous_bool,
                    "accuracy_bool": accuracy_bool,
                    "user_prompt": user_prompt,
                    "generated_problem": generated_problem,
                    "user_answer": user_answer
                    })

with open('print_log.txt', 'w') as w_log:
    print(result['generated_problem'], file = w_log)
    print(result['user_answer'], file = w_log)

In [ ]:
from langchain.prompts import ChatPromptTemplate
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

######## 그래프 상태 값
class MyState(TypedDict): # 그래프의 상태를 정의하는 클래스
    user_question : str
    
graph_builder = StateGraph(MyState) # StateGraph 인스턴스 생성


######## 노드
def test_node_01(state):
    #### LLM
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    
    #### prompt (gpt-4o 23년 10월)
    prompt = ChatPromptTemplate.from_template(
    '''
        - variable
                    Question: {user_question}
                    Context: [{context}]
                    Answer:
        
        test 1을 출력해줘
    ''')
    
    user_question = state["user_question"]
    chain = prompt | llm
    # res = chain.invoke({"context": context, "user_question": user_question})
    res = chain.invoke({"user_question": user_question})
    
    return res.content


def test_node_02(state):
    #### LLM
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    
    #### prompt (gpt-4o 23년 10월)
    prompt = ChatPromptTemplate.from_template(
    '''
        - variable
                    Question: {user_question}
                    Context: [{context}]
                    Answer:
        
        test 1을 출력해줘
    ''')
    
    user_question = state["user_question"]
    chain = prompt | llm
    # res = chain.invoke({"context": context, "user_question": user_question})
    res = chain.invoke({"user_question": user_question})
    
    return res.content


#### 노드 선언
graph_builder.add_node("test_node_01", test_node_01)
graph_builder.add_node("test_node_02", test_node_02)



######## 엣지
#### 엣지 선언
graph_builder.add_edge(START, "test_node_01")
graph_builder.add_edge(START, "test_node_02")
# graph_builder.add_edge("test_node_01", "test_node_02")
graph_builder.add_edge("test_node_01", END)
graph_builder.add_edge("test_node_02", END)



######## 그래프
#### 그래프 컴파일
graph = graph_builder.compile()

#### 그래프 실행
user_question = "기분이 우울한데 책좀 추천해줘"

result = graph.invoke({"user_question": user_question,
                    # "user_answer": user_answer
                    })


서울에는 다양한 맛집이 많아서 선택하기가 쉽지 않지만, 다음은 서울에서 유명한 맛집 TOP 10을 추천해 드립니다.

1. **광장시장** - 전통 시장으로, 빈대떡, 마약김밥, 떡볶이 등 다양한 길거리 음식을 즐길 수 있습니다.

2. **이태원 부대찌개** - 부대찌개로 유명한 이곳은 푸짐한 양과 깊은 맛으로 많은 사랑을 받고 있습니다.

3. **명동교자** - 칼국수와 만두가 유명한 곳으로, 항상 많은 사람들이 줄 서서 기다리는 인기 맛집입니다.

4. **삼청동 수제비** - 수제비와 전통 한식을 맛볼 수 있는 아늑한 분위기의 식당입니다.

5. **한남동 소고기집** - 고급 한우를 맛볼 수 있는 곳으로, 고기 애호가들에게 추천합니다.

6. **홍대 돈부리** - 일본식 덮밥 전문점으로, 다양한 종류의 돈부리를 제공합니다.

7. **압구정 로데오 거리의 카페** - 다양한 디저트와 커피를 즐길 수 있는 세련된 카페들이 많습니다.

8. **신사동 가로수길의 브런치 카페** - 트렌디한 브런치 메뉴를 제공하는 카페들이 많아 인기가 높습니다.

9. **종로의 전통 한정식** - 다양한 반찬과 함께 정갈한 한정식을 즐길 수 있는 곳입니다.

10. **강남의 초밥집** - 신선한 재료로 만든 초밥을 제공하는 고급 초밥집들이 많습니다.

각 맛집마다 특색이 있으니, 방문하실 때 참고하시기 바랍니다!


In [61]:
#### 사용자 이전 질의와의 관련성 확인 필요 유무
#### 사용자 질의의 문법 
def cvt_to_stereotyping(state): # 정형화된 문구로 전환 명령
    user_question = state["state_user_question"]
    system_prompt = f'사용자의 질의 :{user_question}, 사용자의 질의를 정형화된 문구로 변환해줘'
    response_mini = gpt4o_mini.invoke([HumanMessage(content=system_prompt)])
    
    return {"state_user_question": response_mini} # convert user_question


def mem_previous_questions(state): # 이전 질의 n개까지 참고하기 위한 명령 ------- 새로운 상태값을 통해서? 단순히 시스템 프롬프트를 통해서?
    user_question = state["state_user_question"]
    system_prompt = f'사용자의 질의 :{user_question}, 사용자의 질의가 이전 질문들과 연관있는지 확인하고 연관이 있으면 사용자의 질의를 바꿔줘'
    response_mini = gpt4o_mini.invoke([HumanMessage(content=system_prompt)])
    
    return {"state_user_question": response_mini} # convert user_question - 근데 gpt가 알아서 해줄듯? 필요한가?


#### 사용자 질의의 context
def node_name(state): # 특정 웹사이트에서 제공되는 문제를 찾는가
    user_question = state["state_user_question"]
    system_prompt = f'사용자의 질의 :{user_question}, 사용자가 원하는 특정 웹사이트의 이름만 출력해줘'
    response_mini = gpt4o_mini.invoke([HumanMessage(content=system_prompt)])
    
    return {"state_website": response_mini} # generate state_website


#### 코드 평가 결과 명령
def node_name(state): # 평가 방법을 찾지 못했는가
    user_question =_question"]
    system_prompt = f'사용 state["state_user자의 질의 :{user_question}, '
    response_mini = gpt4o_mini.invoke([HumanMessage(content=system_prompt)])
    
    return {"state_user_question": response_mini}


def node_name(state): # 평가 결과가 정확한가
    user_question = state["state_user_question"]
    system_prompt = f'사용자의 질의 :{user_question}, '
    response_mini = gpt4o_mini.invoke([HumanMessage(content=system_prompt)])
    
    return {"state_user_question": response_mini}


def searching_on_DB(state): # DB나 웹사이트에서 데이터를 찾아야하는가
    state["state_db?"]
    user_question = state["state_user_question"]
    system_prompt = f'사용자의 질의 :{user_question}, '
    response_mini = gpt4o_mini.invoke([HumanMessage(content=system_prompt)])
    
    return {"state_user_question": response_mini}

def searching_on_url(state): # DB나 웹사이트에서 데이터를 찾아야하는가
    state["state_context_url"]
    user_question = state["state_user_question"]
    system_prompt = f'사용자의 질의 :{user_question}, '
    response_mini = gpt4o_mini.invoke([HumanMessage(content=system_prompt)])
    
    return {"state_user_question": response_mini}


#### 사용자의 다음 질의 준비
def node_name(state): # 
    user_question = state["state_user_question"]
    system_prompt = f'사용자의 질의{user_question}, '
    response_mini = gpt4o_mini.invoke([HumanMessage(content=system_prompt)])
    
    return {"state_user_question": response_mini}


#### 노드 양식
# def node_name(state): # 
#     user_question = state["state_user_question"]
#     system_prompt = f'사용자의 질의{user_question}, '
#     response_mini = gpt4o_mini.invoke([HumanMessage(content=system_prompt)])
    
#     return {"state_user_question": response_mini}



#### 노드 선언
graph.add_node("node_name", node_name)
graph.add_node("node_name", node_name)
graph.add_node("node_name", node_name)
graph.add_node("node_name", node_name)



In [ ]:
#### 그래프
class MyState(TypedDict): # 그래프의 상태를 정의하는 클래스
    user_question: str
    counter: int
    answer: str

graph = StateGraph(MyState) # StateGraph 인스턴스 생성


#### 노드
def cnt_p_1(state): # 카운터 1 증가
    
    return {"counter": state["counter"] + 1}


def cnt_m_1(state): # 카운터 1 증가
    
    return {"counter": state["counter"] - 1}


def question_to_model(state):
    response_mini = 500
    
    return {"answer": response_mini}


def check_question(state):
    print('condition_function :', check_question, state["answer"])
    if state["answer"] < 50:
        print(50)
        return 50
    elif state["answer"] < 100: 
        print(100)
        return 100
    elif state["answer"] < 200:
        print(200)
        return 200
    else:
        print(999)
        return 999
        


graph.add_node("cnt_p_1", cnt_p_1)
graph.add_node("cnt_m_1", cnt_m_1)
graph.add_node("question_to_model", question_to_model)
graph.add_node("check_question", check_question)


#### 엣지
graph.add_edge(START, "question_to_model")

graph.add_conditional_edges(
                "question_to_model",
                check_question,
                {
                    50: "cnt_p_1",
                    100: "cnt_m_1",
                    200: "cnt_m_1",
                    999: END
                })
